1\. Analtikus függvényes lekérdezés.

Foglalások átlagos tartózkodási ideje szálláshelyenként.

In [8]:
SELECT 
    SZ.SZALLAS_ID,
    SZ.SZALLAS_NEV,
    AVG(DATEDIFF(day, F.METTOL, F.MEDDIG))
FROM SZALLASHELY SZ
JOIN SZOBA S ON SZ.SZALLAS_ID = S.SZALLAS_FK
JOIN FOGLALAS F ON S.SZOBA_ID = F.SZOBA_FK
GROUP BY SZ.SZALLAS_ID, SZ.SZALLAS_NEV;

(12 rows affected)

Total execution time: 00:00:00.082

SZALLAS_ID,SZALLAS_NEV,(No column name)
9,Jáde panzió,4
3,Fortuna Apartman,3
12,Gold Hotel,4
6,Kentaur Hotel,4
7,Szieszta Apartmanház,3
1,Sába-Ház,4
10,Lagúna Hotel,3
4,Fortuna panzió,4
5,Fortuna Panzió,4
2,Családi Ház,3


2\. Analtikus függvényes lekérdezés.

Szállás helyek összesített foglalási száma csillagok szerint.

In [6]:
SELECT 
    SZ.CSILLAGOK_SZAMA,
    COUNT(F.FOGLALAS_PK) AS FOGLALASOK_SZAMA
FROM SZALLASHELY SZ
JOIN SZOBA S ON SZ.SZALLAS_ID = S.SZALLAS_FK
JOIN FOGLALAS F ON S.SZOBA_ID = F.SZOBA_FK
GROUP BY SZ.CSILLAGOK_SZAMA;

(3 rows affected)

Total execution time: 00:00:00.238

CSILLAGOK_SZAMA,FOGLALASOK_SZAMA
0,187
3,513
4,327


1\. Függvény

Visszaadja a megadott szálláshelyhez tartozó foglalások számát a szállás azonosítója alapján.

In [7]:
CREATE FUNCTION FoglalasSzam
(@szallasID NUMERIC(3))
RETURNS INT
AS
BEGIN
    DECLARE @foglalasSzam INT

    SELECT @foglalasSzam = COUNT(*)
    FROM FOGLALAS f
    JOIN SZOBA s ON f.SZOBA_FK = s.SZOBA_ID
    WHERE s.SZALLAS_FK = @szallasID

    RETURN @foglalasSzam
END

: Msg 262, Level 14, State 18, Procedure FoglalasSzam, Line 1
CREATE FUNCTION permission denied in database 'sqlgyak'.

Total execution time: 00:00:00.065

2\. Függvény

Visszaadja az összes foglalás számát.

In [1]:
CREATE FUNCTION osszesFoglalasSzam()
RETURNS INT
AS
BEGIN
    DECLARE @osszesFoglalas INT
    SELECT @osszesFoglalas = COUNT(*) FROM FOGLALAS
    RETURN @osszesFoglalas
END;

: Msg 262, Level 14, State 18, Procedure osszesFoglalasSzam, Line 1
CREATE FUNCTION permission denied in database 'sqlgyak'.

Total execution time: 00:00:00.067

Plusz 1 tábla bővítés és lekérdezés, melyben az új oszlop is szerepel.

A lekérdezés megszámolja, hogy melyik szolgáltatásból mennyi szálláshely található.

In [4]:
-- Új tábla
CREATE TABLE Szolgaltatasok (
    SZOLGALTATAS_ID INT PRIMARY KEY,
    LEIRAS XML,
    KOLTSEG NUMERIC(10, 2),
);

-- Új tábla adatfeltöltése
INSERT INTO SZOLGALTATASOK (SZOLGALTATAS_ID, LEIRAS, KOLTSEG) VALUES
(1, 'Ingyenes wifi', NULL),
(2, 'Reggeli', 4000),
(3, 'Parkoló', 1500),
(4, 'Medence', 6000),
(5, 'Fitneszterem', NULL);

-- Plusz mező a Szálláshely táblában
ALTER TABLE SZALLASHELY
ADD SZOLGALTATAS_FK INT,
FOREIGN KEY (SZOLGALTATAS_FK) REFERENCES SZOLGALTATASOK(SZOLGALTATAS_ID);

-- Plusz mező adatfeltöltése
UPDATE SZALLASHELY
SET SZOLGALTATAS_FK = (SELECT SZOLGALTATAS_ID FROM SZOLGALTATASOK WHERE CAST(LEIRAS as NVARCHAR(20)) = 'Fitneszterem')
WHERE TIPUS = 'panzió';

UPDATE SZALLASHELY
SET SZOLGALTATAS_FK = (SELECT SZOLGALTATAS_ID FROM SZOLGALTATASOK WHERE CAST(LEIRAS as NVARCHAR(20)) = 'Medence')
WHERE TIPUS = 'Hotel';

UPDATE SZALLASHELY
SET SZOLGALTATAS_FK = (SELECT SZOLGALTATAS_ID FROM SZOLGALTATASOK WHERE CAST(LEIRAS as NVARCHAR(20)) = 'Reggeli')
WHERE TIPUS = 'vendégház';

UPDATE SZALLASHELY
SET SZOLGALTATAS_FK = (SELECT SZOLGALTATAS_ID FROM SZOLGALTATASOK WHERE CAST(LEIRAS as NVARCHAR(20)) = 'Ingyenes wifi')
WHERE TIPUS = 'Diákszálló';

UPDATE SZALLASHELY
SET SZOLGALTATAS_FK = (SELECT SZOLGALTATAS_ID FROM SZOLGALTATASOK WHERE CAST(LEIRAS as NVARCHAR(20)) = 'Parkoló')
WHERE TIPUS = 'Apartman';

-- Lekérdezés
SELECT 
    CAST(S.LEIRAS as NVARCHAR(20)),
    COUNT(SH.TIPUS) AS SZALLASOK_SZAMA
FROM Szolgaltatasok S
JOIN Szallashely SH ON S.SZOLGALTATAS_ID = SH.SZOLGALTATAS_FK
GROUP BY CAST(S.LEIRAS as NVARCHAR(20));

: Msg 1088, Level 16, State 13, Line 1
Cannot find the object "SZALLASHELY" because it does not exist or you do not have permissions.

Total execution time: 00:00:00.636